In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [6]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2 = df.copy()
df2['loan_status'] = le.fit_transform(df2['loan_status'])
df2['issue_d'] = le.fit_transform(df2['issue_d'])
df2['next_pymnt_d'] = le.fit_transform(df2['next_pymnt_d'])
df2 = pd.get_dummies(df2, columns=["home_ownership", "verification_status", "hardship_flag", "debt_settlement_flag", "pymnt_plan", "initial_list_status", "application_type"])
df2.head()

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,hardship_flag_N,debt_settlement_flag_N,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,2,1,27.24,0.0,0.0,8.0,...,0,1,0,1,1,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,2,1,20.23,0.0,0.0,17.0,...,0,0,1,1,1,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,2,1,24.26,0.0,0.0,8.0,...,0,0,1,1,1,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,2,1,31.44,0.0,1.0,10.0,...,0,0,1,1,1,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,2,1,18.76,0.0,1.0,14.0,...,1,0,0,1,1,1,0,1,1,0


In [8]:
# Create our features
X = df2.drop(columns=["loan_status"])

# Create our target
y = df2["loan_status"]

In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,hardship_flag_N,debt_settlement_flag_N,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,0.805542,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.478007,0.373992,0.148001,1.0,1.0,1.0,0.123879,0.876121,0.860340,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,0.714932,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.499520,0.483865,0.355104,0.0,0.0,0.0,0.329446,0.329446,0.346637,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,0.000000,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,1.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,1.000000,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,1.000000,26.660000,0.000000,1.000000,16.000000,0.000000,...,1.000000,1.000000,0.000000,1.0,1.0,1.0,0.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,2.000000,999.000000,18.000000,5.000000,72.000000,4.000000,...,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000


In [11]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

clf = BalancedRandomForestClassifier(random_state=1, n_estimators=100) \
                .fit(X_train, y_train)
print(f'Training Score: {clf.score(X_train, y_train)}')
print(f'Testing Score: {clf.score(X_test, y_test)}')
clf.fit(X, y)

Training Score: 0.8541811981709679
Testing Score: 0.8478930543446672


BalancedRandomForestClassifier(random_state=1)

In [16]:
# Calculated the balanced accuracy score
y_pred = clf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9389324134705332

In [17]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  101,     0],
       [ 2089, 15015]])

In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.05      1.00      0.88      0.09      0.94      0.89       101
          1       1.00      0.88      1.00      0.93      0.94      0.87     17104

avg / total       0.99      0.88      1.00      0.93      0.94      0.87     17205



In [51]:
importances = clf.feature_importances_
importances


array([0.01328516, 0.03026769, 0.01856521, 0.01634431, 0.02469675,
       0.02014948, 0.00378107, 0.00506867, 0.00911336, 0.00133303,
       0.01439362, 0.0103423 , 0.01705421, 0.01548631, 0.05870063,
       0.04080006, 0.09005781, 0.05423157, 0.00784983, 0.        ,
       0.        , 0.06272953, 0.00678691, 0.000583  , 0.        ,
       0.        , 0.00454012, 0.01450356, 0.00586262, 0.00571346,
       0.00452003, 0.00765639, 0.0142454 , 0.01091303, 0.01349645,
       0.0039961 , 0.00839446, 0.01525605, 0.01279834, 0.01422709,
       0.00649389, 0.00643411, 0.0081556 , 0.00879936, 0.01380768,
       0.01405392, 0.01453529, 0.00024317, 0.        , 0.0154663 ,
       0.01345116, 0.01151123, 0.00915959, 0.00610549, 0.01402464,
       0.01813708, 0.00461018, 0.00801955, 0.00939281, 0.00699577,
       0.01054528, 0.00993398, 0.00948921, 0.01042058, 0.00839681,
       0.00844494, 0.        , 0.        , 0.00081534, 0.00642347,
       0.00879895, 0.00627066, 0.00160365, 0.        , 0.01323

In [53]:
sorted(zip(clf.feature_importances_, X.columns), reverse=True)

[(0.09005781294752832, 'total_rec_prncp'),
 (0.06272952565266024, 'last_pymnt_amnt'),
 (0.05870062853882819, 'total_pymnt'),
 (0.05423156612782609, 'total_rec_int'),
 (0.04080005963428618, 'total_pymnt_inv'),
 (0.030267688596143056, 'int_rate'),
 (0.024696754414855204, 'issue_d'),
 (0.020149483790754995, 'dti'),
 (0.018565208545976285, 'installment'),
 (0.018137083423474473, 'mths_since_recent_inq'),
 (0.01705420867510259, 'out_prncp'),
 (0.016461919140855465, 'total_bc_limit'),
 (0.016344306054746096, 'annual_inc'),
 (0.01548631254394858, 'out_prncp_inv'),
 (0.015466298595178203, 'mo_sin_old_il_acct'),
 (0.015256050515618458, 'max_bal_bc'),
 (0.014535292250851671, 'bc_util'),
 (0.014503555489585424, 'tot_cur_bal'),
 (0.014393618051483877, 'revol_bal'),
 (0.014245403173020275, 'mths_since_rcnt_il'),
 (0.014227092918517092, 'total_rev_hi_lim'),
 (0.014053919955408114, 'bc_open_to_buy'),
 (0.01402464362022519, 'mths_since_recent_bc'),
 (0.013807684171255956, 'avg_cur_bal'),
 (0.013533115

### Easy Ensemble AdaBoost Classifier

In [54]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1) \
                .fit(X_train, y_train)

eec.fit(X, y)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [55]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9652217881984644

In [56]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   99,     2],
       [  851, 16253]])

In [57]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.10      0.98      0.95      0.19      0.97      0.93       101
          1       1.00      0.95      0.98      0.97      0.97      0.93     17104

avg / total       0.99      0.95      0.98      0.97      0.97      0.93     17205

